In [1]:
import pyarrow.parquet as pq
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from pathlib import Path

In [2]:
%reload_ext autoreload
%autoreload 2
#ignore warnings.
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

#imports
import pandas as pd
import os, sys
import importlib
import yaml, json
from pathlib import Path
from sklearn.model_selection import train_test_split
#path append
sys.path.append(os.path.join(Path.cwd(), './modules'))
import Evaluate, Helper, Present, Score, Synthetic, Modeling


In [3]:
#multiple prediction file load still works, just loaded a single file because too many things are being displayed otherwise.
config_file = './config/config.yaml'
prediction_config_files = ['./config/predictions/pred1.yaml']
c_r, c_e, c_gen, c_aws, c_visual, predictions=Helper.load_configuration(config_file, prediction_config_files)

In [4]:

base_dir = Path.cwd() #For running locally
#base_dir = cwd_dir.parent
data_dir= base_dir / 'data' /'simulated_health_data' / 'data' 


Let's provide some examples of working with partitioned parquet files. 

In [5]:
health=pd.read_csv(data_dir / 'withtimeseries-10-100-10-5-2-True-10-5-2-True-2-101-5-0-0.csv')
health

,person_id,yyyymm,cad0,cad1,cad2,cad3,cad4,cad5,cad6,cad7,...,dv4,dv5,dv6,dv7,dv8,dv9,target_bin,target_class,target,index_c
0,1000,201701,1,1,1,1,1,1,1,1,...,0.651118,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0
1,1000,201702,1,1,1,1,1,1,1,1,...,0.651118,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0
2,1000,201703,1,1,1,1,1,1,1,1,...,0.651118,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0
3,1000,201704,1,1,1,1,1,1,1,1,...,0.651118,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0
4,1000,201705,1,1,1,1,1,1,1,1,...,0.651118,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215995,9999,201808,1,1,1,0,0,0,0,0,...,0.487144,-0.686401,-0.218947,0.273039,0.011350,-0.370388,0,0,-27.788632,99
215996,9999,201809,1,1,1,0,0,0,0,0,...,0.487144,-0.686401,-0.218947,0.273039,0.011350,-0.370388,0,0,-27.788632,99
215997,9999,201810,1,1,1,0,0,0,0,0,...,0.487144,-0.686401,-0.218947,0.273039,0.011350,-0.370388,0,0,-27.788632,99
215998,9999,201811,1,1,1,0,0,0,0,0,...,0.487144,-0.686401,-0.218947,0.273039,0.011350,-0.370388,0,0,-27.788632,99


In [6]:
referrals=Helper.read_file(directory=c_r['dir'],file=c_r['file'],file_format=c_r['file_format'],aws=c_r['aws'],bucket= c_r['bucket'])
referrals

,PERSON_ID,MYR,REASON
0,30,1/29/17,Diet
1,53,5/17/16,Kidney
2,120,2/27/16,Lung
3,154,8/31/16,Diet
4,158,8/29/16,Kidney
5,204,9/29/17,Lung
6,233,8/10/17,Diet
7,279,11/27/16,Kidney
8,294,8/20/16,Lung
9,325,4/19/17,Diet


In [7]:
#Need to match the Health data with the referral.  
#NAFIS TODO
#Prior referrals should be incorporated as features. 
#Should be able to use similar logic to the evaluation merge. 1 if it becomes referral in any months in the next window. 
#Instead going to continue 
#I think we should try to do 1 month for training and another month for testing. 

In [8]:
#select out one month and set index equal to the personid
health=health.loc[health['yyyymm']==201712,:]
health.index=health['person_id']
health

,person_id,yyyymm,cad0,cad1,cad2,cad3,cad4,cad5,cad6,cad7,...,dv4,dv5,dv6,dv7,dv8,dv9,target_bin,target_class,target,index_c
person_id,,,,,,,,,,,,,,,,,,,,,
1000,1000,201712,1,1,1,1,1,1,1,1,...,0.651118,-0.319318,-0.848077,0.605965,-2.018168,0.740122,1,1,305.032212,0
1001,1001,201712,1,1,1,1,1,1,1,1,...,-0.933237,0.955057,0.190794,1.978757,2.605967,0.683509,0,0,-84.698542,1
1002,1002,201712,1,1,1,1,1,1,1,1,...,-0.134841,0.390528,0.166905,0.184502,0.807706,0.072960,0,0,-0.915500,2
1003,1003,201712,1,1,1,1,1,1,1,1,...,-0.943406,0.484752,-0.116773,1.901755,0.238127,1.996652,0,0,-51.293606,3
1004,1004,201712,1,1,1,1,1,1,1,1,...,1.025984,-0.156598,-0.031579,0.649826,2.154846,-0.610259,0,0,-62.943629,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,201712,1,1,1,0,0,0,0,0,...,-0.814544,0.389736,0.389634,-0.012625,0.161977,1.499623,0,0,-5.806354,95
9996,9996,201712,1,1,1,0,0,0,0,0,...,-0.308765,-0.265327,-0.424594,1.868926,1.437781,-1.389828,0,0,0.429037,96
9997,9997,201712,1,1,1,0,0,0,0,0,...,-0.121841,0.190861,0.432215,-0.731353,-0.075012,2.529188,0,0,-153.759155,97


In [9]:
#TRAINTEST SPLIT
y=health['target_bin']
y_class=health['target_class']
X=health[['cad0', 'cad1', 'cad2', 'cad3', 'cad4', 'cad5',
       'cad6', 'cad7', 'cad8', 'cad9',  'dv0', 'dv1', 'dv2', 'dv3',
       'dv4', 'dv5', 'dv6', 'dv7', 'dv8', 'dv9']]

In [10]:
train_X, test_X, train_y, test_y = train_test_split(X, y, 
                                                    train_size=0.5,
                                                    test_size=0.5,
                                                    random_state=123,
                                                    stratify=y)

In [11]:
train_predict, test_predict, trained_models= Modeling.perform_classification(train_X, test_X, train_y, test_y)

Fitting model:  k_nearest_neighbors_10
Fitting model:  svm_linear
Fitting model:  svm_nonlinear
Fitting model:  decision_tree
Fitting model:  naive_bayes
Fitting model:  random_forest
Fitting model:  adaboost
Fitting model:  quadratic_discriminant_analysis


In [12]:
test_predict

,target,k_nearest_neighbors_10,svm_linear,svm_nonlinear,decision_tree,naive_bayes,random_forest,adaboost,quadratic_discriminant_analysis
person_id,,,,,,,,,
1635,0,0.0,0.000638,2.306288e-04,0.0,0.005911,0.1,0.442895,5.334624e-05
1682,1,0.3,0.732355,1.000000e+00,1.0,0.428256,0.9,0.517489,8.690149e-01
4285,1,0.6,0.633928,1.000000e+00,1.0,0.440287,1.0,0.496106,7.959353e-01
2661,0,0.0,0.001116,1.000000e-07,0.0,0.026120,0.0,0.459586,2.532655e-05
4553,0,0.0,0.007796,8.064559e-05,0.0,0.062092,0.0,0.470665,4.330735e-03
...,...,...,...,...,...,...,...,...,...
8792,0,0.0,0.001415,1.000000e-07,0.0,0.016670,0.0,0.447351,7.303137e-05
5843,1,0.9,0.543065,9.999997e-01,1.0,0.368506,1.0,0.508206,6.541873e-01
2762,0,0.0,0.048442,3.702401e-01,0.0,0.116714,0.0,0.481844,3.609189e-02
